In [2]:
!pip install requests pandas
import requests
import pandas as pd
import time
import datetime

# 24시간마다 갱신해야 하는 키입니다.
API_KEY = "RGAPI-###"
REGION = "kr"

# 추적 대상 플레이어 목록
# Nickname : ["Encrypted Summoner ID", Original ELO, "Team name", Original # of matches ]
PLAYERS_TO_TRACK = {
    "###": ["###", 2000, "Team 1", 0],
}
# --- 설정 영역 끝 ---

def display_team_rank_change_status():
    """
    현재 솔랭 점수를 조회하고, 기준 통합 점수 대비 등락폭을 계산한 후,
    팀별로 등락폭 합계를 구하여 출력합니다.
    """

    tier_dict = dict([["언랭크", 0], ["IRON", 0], ["BRONZE", 400], ["SILVER", 800], ["GOLD", 1200], ["PLATINUM", 1600], ["EMERALD", 2000], ["DIAMOND", 2400], ["MASTER", 2800]])
    div_dict = dict([["I", 300], ["II", 200], ["III", 100], ["IV", 0]])

    current_records = []

    # 팀별 등락폭 합산 변수 초기화
    team_0_total_change, team_1_total_change = 0, 0

    # 딕셔너리에서 이름, ID, 기준 점수, 팀 ID를 분리하여 반복
    for name, data in PLAYERS_TO_TRACK.items():
        summoner_id, ref_score, team_id, ref_games = data[0], data[1], data[2], data[3]

        try:
            # API 호출
            url = f"https://{REGION}.api.riotgames.com/lol/league/v4/entries/by-puuid/{summoner_id}"
            headers = {"X-Riot-Token": API_KEY}
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                api_data = response.json()
                solo_rank = next((e for e in api_data if e.get('queueType') == 'RANKED_SOLO_5x5'), None)

                if solo_rank:
                    current_score = solo_rank['leaguePoints'] + tier_dict.get(solo_rank['tier'], -10000) + div_dict.get(solo_rank['rank'], -10000)
                    num_games = solo_rank['wins'] + solo_rank['losses'] - ref_games
                    score_change_from_ref = current_score - ref_score + 2 * num_games

                    if team_id == "Team 1":
                        team_0_total_change += score_change_from_ref
                    elif team_id == "Team 2":
                        team_1_total_change += score_change_from_ref

                    current_records.append({
                        '닉네임': name,
                        '팀': team_id,
                        '기준 점수': ref_score,
                        '현재 점수': current_score,
                        '점수 변동': f"{score_change_from_ref:+d}",
                        '현재 티어': f"{solo_rank['tier']} {solo_rank['rank']} ({solo_rank['leaguePoints']} LP)",
                        '판수': num_games
                    })
                else:
                    score_change_from_ref = 0 - ref_score
                    if team_id == "Team 1":
                        team_0_total_change += score_change_from_ref
                    elif team_id == "Team 2":
                        team_1_total_change += score_change_from_ref

                    current_records.append({
                        '닉네임': name, '팀': team_id, '기준 점수': ref_score, '현재 점수': 0,
                        '점수 변동': f"{score_change_from_ref:+d}", '현재 티어': "언랭크", '판수': 0
                    })

            elif response.status_code == 403:
                print(f"❌ 오류: Riot API Key 만료 (403). 키를 갱신하세요!")
                return

            else:
                print(f"⚠️ 오류: {name} 정보 조회 실패. 상태 코드: {response.status_code}")

        except Exception as e:
            print(f"❌ 코드 실행 오류 ({name}): {e}")

        time.sleep(0.3)

    # 4. 개인별 결과 출력
    if current_records:
        df_result = pd.DataFrame(current_records)

        # 통합 점수 변동에 따라 정렬
        df_result['Score_Change_Numeric'] = df_result['점수 변동'].str.replace('+', '').astype(int)
        df_result_sorted_by_change = df_result.sort_values(by='Score_Change_Numeric', ascending=False).drop(columns=['Score_Change_Numeric'])

        columns_order = ['닉네임', '팀', '현재 티어', '판수', '현재 점수', '기준 점수', '점수 변동']
        df_final = df_result_sorted_by_change[columns_order]

        print("\n--- ✅ 개인별 통합 점수 변동 현황 테이블 ✅ ---")
        print(df_final.to_markdown(index=False))
        print("\n----------------------------------------------------")
    else:
        print("\n조회된 플레이어 정보가 없습니다. API Key 및 ID 설정을 확인해 주세요.")

    # 5. ⚠️ 최종 팀별 점수 합산 결과 출력 ⚠️
    print("\n====================================================")
    print("                🏆 팀별 등락폭 합산 결과 🏆")
    print("====================================================")
    print(f"팀 1 등락폭 합계: {team_0_total_change:+d} 점")
    print(f"팀 2 등락폭 합계: {team_1_total_change:+d} 점")
    print("====================================================")
display_team_rank_change_status()

⚠️ 오류: ### 정보 조회 실패. 상태 코드: 401

조회된 플레이어 정보가 없습니다. API Key 및 ID 설정을 확인해 주세요.

                🏆 팀별 등락폭 합산 결과 🏆
팀 1 등락폭 합계: +0 점
팀 2 등락폭 합계: +0 점
